In [19]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [20]:
# network parameters
N_assets = 50
n_asset_index = 2
n_macro_index = 8
# n_Moment_RNN = 4

In [21]:
# State RNN
inputs = keras.Input(shape=(n_macro_index))
n_macro_info = 4   # output dimension
n_samples = tf.shape(inputs)[0]
inputs0 = tf.reshape(inputs,[1,n_samples,n_macro_index]) # reshape it as a sequence

State_RNN1 = layers.LSTM(n_macro_info,return_sequences = True)
outputs0 = State_RNN1(inputs0)

outputs = tf.reshape(outputs0,[n_samples,n_macro_info])
State_RNN = keras.Model(inputs=inputs, outputs=outputs, name = 'State_RNN')
State_RNN.summary()
# inputs = keras.Input(shape=(n_macro_index,))
# outputs = State_RNN(inputs)
# test = keras.Model(inputs = inputs, outputs = outputs, name = 'test')
# test.summary()

Model: "State_RNN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 8)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_Shape_7 (TensorFlow [(2,)]               0           input_18[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_strided_slice_7 (Te [()]                 0           tf_op_layer_Shape_7[0][0]        
__________________________________________________________________________________________________
tf_op_layer_Reshape_9/shape (Te [(3,)]               0           tf_op_layer_strided_slice_7[0][0]
__________________________________________________________________________________________

In [6]:
# State FFN model
inputs = keras.Input(shape = (n_asset_index + n_macro_info))
State_FFN1 = layers.Dense(32, activation = 'relu')
State_FFN2 = layers.Dense(32, activation = 'relu')
State_FFN3 = layers.Dense(1, activation = 'linear')
outputs = State_FFN1(inputs)
outputs = State_FFN2(outputs)
outputs = State_FFN3(outputs)
State_FFN = keras.Model(inputs=inputs, outputs=outputs, name = 'State_FFN')
State_FFN.summary()

Model: "State_FFN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 6)]               0         
_________________________________________________________________
dense (Dense)                (None, 32)                224       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,313
Trainable params: 1,313
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Global State FFN model

inputs1 = keras.Input(shape = (N_assets, n_asset_index))
n_samples1 = tf.shape(inputs1)[0]
inputs10 = tf.reshape(inputs1,[n_samples1*N_assets,n_asset_index])

inputs2 = keras.Input(shape = (n_macro_info))
n_samples2 = tf.shape(inputs2)[0]
inputs20 = tf.tile(inputs2,[N_assets,1])

inputs = [inputs1, inputs2]

inputs0 = tf.concat([inputs10,inputs20],1)
outputs0 = State_FFN(inputs0)
outputs = tf.reshape(outputs0,[n_samples1,N_assets])

Global_State_FFN = keras.Model(inputs=inputs, outputs=outputs, name = 'Global_State_FFN')
Global_State_FFN.summary()

Model: "Global_State_FFN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 50, 2)]      0                                            
__________________________________________________________________________________________________
tf_op_layer_Shape_1 (TensorFlow [(3,)]               0           input_3[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [()]                 0           tf_op_layer_Shape_1[0][0]        
__________________________________________________________________________________________________
tf_op_layer_mul (TensorFlowOpLa [()]                 0           tf_op_layer_strided_slice_1[0][0]
___________________________________________________________________________________

In [8]:
# State 
macro_index = keras.Input(shape=(n_macro_index))
asset_index = keras.Input(shape=(N_assets, n_asset_index))
R = keras.Input(shape=(N_assets))
inputs = [macro_index, asset_index, R]
macro_info = State_RNN(macro_index)
w = Global_State_FFN([asset_index, macro_info])
M = tf.multiply(R,w)
M = tf.reduce_sum(M, 1)
M = 1 - M
State = keras.Model(inputs=inputs, outputs=M, name = 'State')
State.summary()
# output has shape (time_steps, 1)

Model: "State"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 50, 2)]      0                                            
__________________________________________________________________________________________________
State_RNN (Model)               multiple             208         input_5[0][0]                    
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 50)]         0                                            
______________________________________________________________________________________________

In [9]:
# Moment RNN
inputs = keras.Input(shape=(n_macro_index))
n_macro_info = 4   # Moment output dimension
n_samples = tf.shape(inputs)[0]
inputs0 = tf.reshape(inputs,[1,n_samples,n_macro_index]) # reshape it as a sequence
Moment_RNN1 = layers.LSTM(n_macro_info,return_sequences = True)
outputs0 = Moment_RNN1(inputs0)
outputs = tf.reshape(outputs0,[n_samples,n_macro_info])
Moment_RNN = keras.Model(inputs=inputs, outputs=outputs, name = 'Moment_RNN')
Moment_RNN.summary()


Model: "Moment_RNN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_Shape_3 (TensorFlow [(2,)]               0           input_8[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_3 (Te [()]                 0           tf_op_layer_Shape_3[0][0]        
__________________________________________________________________________________________________
tf_op_layer_Reshape_4/shape (Te [(3,)]               0           tf_op_layer_strided_slice_3[0][0]
_________________________________________________________________________________________

In [10]:
# Moment FFN model
inputs = keras.Input(shape = (n_asset_index + n_macro_info))
Moment_FFN1 = layers.Dense(32, activation = 'relu')
Moment_FFN2 = layers.Dense(32, activation = 'relu')
Moment_FFN3 = layers.Dense(1, activation = 'linear')
outputs = Moment_FFN1(inputs)
outputs = Moment_FFN2(outputs)
outputs = Moment_FFN3(outputs)
Moment_FFN = keras.Model(inputs=inputs, outputs=outputs, name = 'Moment_FFN')
State_FFN.summary()

Model: "State_FFN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 6)]               0         
_________________________________________________________________
dense (Dense)                (None, 32)                224       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,313
Trainable params: 1,313
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Global Moment FFN model

inputs1 = keras.Input(shape = (N_assets, n_asset_index))
n_samples1 = tf.shape(inputs1)[0]
inputs10 = tf.reshape(inputs1,[n_samples1*N_assets,n_asset_index])

inputs2 = keras.Input(shape = (n_macro_info))
n_samples2 = tf.shape(inputs2)[0]
inputs20 = tf.tile(inputs2,[N_assets,1])

inputs = [inputs1, inputs2]

inputs0 = tf.concat([inputs10,inputs20],1)
outputs0 = Moment_FFN(inputs0)
outputs = tf.reshape(outputs0,[n_samples1,N_assets])

Global_Moment_FFN = keras.Model(inputs=inputs, outputs=outputs, name = 'Global_Moment_FFN')
Global_Moment_FFN.summary()

Model: "Global_Moment_FFN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 50, 2)]      0                                            
__________________________________________________________________________________________________
tf_op_layer_Shape_4 (TensorFlow [(3,)]               0           input_10[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_strided_slice_4 (Te [()]                 0           tf_op_layer_Shape_4[0][0]        
__________________________________________________________________________________________________
tf_op_layer_mul_2 (TensorFlowOp [()]                 0           tf_op_layer_strided_slice_4[0][0]
__________________________________________________________________________________

In [12]:
# Moment
macro_index = keras.Input(shape=(n_macro_index))
asset_index = keras.Input(shape=(N_assets, n_asset_index))
inputs = [macro_index, asset_index]
macro_info = Moment_RNN(macro_index)
g = Global_Moment_FFN([asset_index, macro_info])
# output has shape (time_steps, N_assets)
Moment = keras.Model(inputs=inputs, outputs=g, name = 'Moment')

Moment.summary()

Model: "Moment"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           [(None, 50, 2)]      0                                            
__________________________________________________________________________________________________
Moment_RNN (Model)              multiple             208         input_12[0][0]                   
__________________________________________________________________________________________________
Global_Moment_FFN (Model)       multiple             1313        input_13[0][0]                   
                                                                 Moment_RNN[1][0]            

In [13]:
# Combined 
macro_index = keras.Input(shape=(n_macro_index))
asset_index = keras.Input(shape=(N_assets, n_asset_index))
R = keras.Input(shape=(N_assets))
print(macro_index.shape, asset_index.shape, R.shape)


inputs = [macro_index, asset_index, R]
M = State([macro_index, asset_index, R])
samples = tf.shape(M)[0]
M = tf.reshape(M,[samples,1])
M_tiled = tf.tile(M,[1,N_assets])

inputs0 = [macro_index, asset_index]
g = Moment(inputs0)

outputs = tf.multiply(g,R)
outputs = tf.multiply(outputs,M_tiled)

Combined = keras.Model(inputs=inputs, outputs=M_tiled, name = 'Combined')
Combined.summary()
# output has shape (time_steps, 1)
# keras.utils.plot_model(Combined, 'my_first_model.png')

(None, 8) (None, 50, 2) (None, 50)
Model: "Combined"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           [(None, 50, 2)]      0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           [(None, 50)]         0                                            
__________________________________________________________________________________________________
State (Model)                   (None,)              1521        input_14[0][0]                   
                                                        

In [14]:
# test
# macro_index = keras.Input(shape=(n_macro_index))
# asset_index = keras.Input(shape=(N_assets, n_asset_index))
R = keras.Input(shape=(N_assets))
# inputs = [macro_index, asset_index, R]
# outputs = [macro_index, asset_index, R]
outputs = keras.layers.Dense(N_assets)(R)
test = keras.Model(inputs=R, outputs=outputs, name = 'test')
# test.summary()

In [22]:
# Numerical Experiments
N_assets = 50
T_train = 25
T_valid = 10
T_test = 25
T = 60
n_asset_index = 2
n_macro_index = 8
np.random.seed(7)
beta = np.random.normal(size = (T, N_assets)) * np.random.normal(size = (T, N_assets))
F = 1 + np.random.normal(size = (T,1))
F = np.tile(F,(1, N_assets))
epsilon = np.random.normal(size = (N_assets,T))*0.5
R = beta*F #+ epsilon
macro_index = np.zeros(shape = (T, n_macro_index))
asset_index = np.zeros(shape = (T, N_assets, n_asset_index))
print(macro_index.shape, asset_index.shape, R.shape)
x_train = [macro_index, asset_index, R]
y_train = np.zeros(shape = (T, N_assets))

(60, 8) (60, 50, 2) (60, 50)


In [16]:
# 这里的compile随便写的compile过程，为了看看整个model对不对，Loss需要改成L2的加权Loss
# 另外训练也要用GAN的方法分别minmax这样训练，朋友，加油
def custom_loss(y_true,y_pred):
    z = y_pred - y_true
    z_abs = tf.abs(z)
    tf.reduce_sum(z_abs, [0, 1])
    return z
Combined.compile(optimizer='adam',loss=custom_loss)

In [18]:
Combined.fit(x=x_train,y=np.zeros(shape= (T,N_assets)), epochs=100,verbose=True)

Train on 60 samples
Epoch 1/100
60/60 [==============================] - 0s 2ms/sample - loss: 0.9869
Epoch 2/100
60/60 [==============================] - 0s 2ms/sample - loss: 0.9873
Epoch 3/100
60/60 [==============================] - 0s 2ms/sample - loss: 0.9866
Epoch 4/100
60/60 [==============================] - 0s 2ms/sample - loss: 0.9866
Epoch 5/100
60/60 [==============================] - 0s 2ms/sample - loss: 0.9867
Epoch 6/100
60/60 [==============================] - 0s 2ms/sample - loss: 0.9863
Epoch 7/100
60/60 [==============================] - 0s 2ms/sample - loss: 0.9861
Epoch 8/100
60/60 [==============================] - 0s 3ms/sample - loss: 0.9860
Epoch 9/100
60/60 [==============================] - 0s 2ms/sample - loss: 0.9859
Epoch 10/100
60/60 [==============================] - 0s 2ms/sample - loss: 0.9865
Epoch 11/100
60/60 [==============================] - 0s 3ms/sample - loss: 0.9856
Epoch 12/100
60/60 [==============================] - 0s 3ms/sample - loss: 

60/60 [==============================] - 0s 2ms/sample - loss: 0.9739
Epoch 100/100
60/60 [==============================] - 0s 3ms/sample - loss: 0.9738
